In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy import ndimage
from func import *
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
import numpy as np
import astropy.io.fits as fits
from astropy.table import Table, hstack
import os
import glob
from astropy.table import vstack, join

In [2]:
def wavlim(vel_OIII,vel_OIII_br):
    c = 300000 # km/s
    k_OIII = 1+(vel_OIII/c)
    k_OIII_br = 1+(vel_OIII_br/c)
    wav_min = (5007*k_OIII_br) - 100
    wav_max = (5007*k_OIII) + 100
    return wav_min,wav_max

def line_shape_par(z,wave,cor_popt):
    c = 3*(10**5)
    cumsum = np.cumsum(OIII_wo_cont(wave,*cor_popt))
    norm_sum=cumsum/cumsum[-1]

    select = (norm_sum>0.1) & (norm_sum<0.9)
    w80_spec = wave[select][-1]-wave[select][0]
    
    w80 = ((w80_spec)/5006.8)*(c/(1+z))   
    return w80


In [3]:
hdu_tab = fits.open('/home/mainak/Downloads/Outflow_paper1/ALPAKA_v1.fits')
data = hdu_tab[1].data
l = len(data)
hdu_tab.close()

In [4]:
for i in range (l):
    print (i)
    hdu = fits.open('/home/mainak/Downloads/Outflow_paper1/ALPAKA_v1.fits')
    data = hdu[1].data
    OIII_5007_flux = data['OIII_5007_flux'][i]
    OIII_5007_br_flux = data['OIII_5007B_flux'][i]
    z = data['z'][i]
    vel_off_OIII_5007 = data['OIII_5007_vel'][i]
    vel_off_OIII_5007_br = data['OIII_5007B_vel'][i]
    fwhm_OIII_5007 = data['OIII_5007_FWHM'][i]
    fwhm_OIII_5007_br = data['OIII_5007B_FWHM'][i]
    L_OIII = data['OIII_5007_LUM'][i]
    L_OIII_br = data['OIII_5007B_LUM'][i]
    L_BLR_Hb = data['HBBB_LUM'][i]
    hdu.close()

    vel_syst = 300000*z
    vel_OIII = vel_syst - vel_off_OIII_5007
    vel_OIII_br = vel_syst - vel_off_OIII_5007_br
    vel_sigma_OIII = fwhm_OIII_5007/2.3548
    vel_sigma_OIII_br = fwhm_OIII_5007_br/2.3548
    sigma_OIII_5007 = line_width_recons(vel_sigma_OIII,rest_line=5006.8,inst_res_fwhm=0)
    sigma_OIII_5007_br = line_width_recons(vel_sigma_OIII_br,rest_line=5006.8,inst_res_fwhm=0) 
    amp_OIII5007 = OIII_5007_flux/(np.sqrt(2.*np.pi)*np.fabs(sigma_OIII_5007))
    amp_OIII5007_br = OIII_5007_br_flux/(np.sqrt(2.*np.pi)*np.fabs(sigma_OIII_5007_br))

    if vel_sigma_OIII_br == 0:
        log_lum_OIII = np.log10(L_OIII)
        w80 = 1.4*fwhm_OIII_5007
    else:
        cor_popt = np.array([amp_OIII5007,amp_OIII5007_br,vel_OIII,vel_sigma_OIII,vel_OIII_br,vel_sigma_OIII_br])
        (wav_min,wav_max) = (wavlim(vel_OIII,vel_OIII_br))
        wave = (np.arange(wav_min,wav_max,0.02))    
        w80 = line_shape_par(z,wave,cor_popt)
        log_lum_OIII = np.log10(L_OIII+L_OIII_br)
    log_lum_Hb_blr = np.log10(L_BLR_Hb)
    print (w80,log_lum_OIII,log_lum_Hb_blr)
    
    col1 = fits.Column(name='source no', format='E', array=[i])
    col2 = fits.Column(name='w80', format='E', array=[w80])
    col3 = fits.Column(name='log_L_OIII', format='E', array=[log_lum_OIII])
    col4 = fits.Column(name='log_L_Hb_blr', format='E', array=[log_lum_Hb_blr])
    t = fits.BinTableHDU.from_columns([col1, col2, col3, col4])
    t.writeto('/home/mainak/Downloads/Outflow_paper1/Mullaney Spectral Parameters/%s_source.fits'%(i),overwrite=True)

0
386.8769976030343 41.539588391522074 -inf
1


/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in log10
/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


442.57982788085934 40.151809174562835 39.52834331052045
2
541.711574665419 41.19468991414004 -inf
3
466.4585236229679 41.15968655938938 40.537508934432054
4
447.3578260639523 41.30155680715276 41.05012113629107
5
348.1330411742104 42.22276356569653 42.11312886465226
6
353.02614548823635 40.01600935073576 -inf
7
469.8677182337573 41.08658963525854 -inf
8
348.9167843914373 40.84151603009184 39.87242648024541
9
976.4229248046875 40.201475073776514 39.67415518522964
10
819.572509765625 40.57474520220503 -inf
11
532.2084767377557 41.62024735535049 41.94402313210027
12
487.91894089243794 41.31830954119661 40.46204657863444
13
399.8755163642564 41.61313581181216 41.34287790128794
14
442.6752714669655 42.396724247435046 41.7318151515955
15
711.4758728027343 -inf -inf
16


/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log10


752.1890803175882 41.33106575616263 39.92944307165836
17
345.62230834960934 40.79015510786439 40.07581027429695
18
271.9399810791015 40.34260874617762 23.59761942165911
19
281.9529406714982 41.2930111743737 -inf
20
756.9527208408989 41.585589800504785 40.61910132077682
21
466.97611914088566 41.148757001679506 41.31485282662267
22
916.390636252316 41.41701401789929 41.77663438810431
23
670.3867577569874 40.88869575382916 -inf
24
578.1239660417893 41.0720991300347 -inf
25
507.1026811652585 42.20923351250419 42.153003487545874
26
312.83153381347654 39.99428153205592 -inf
27
341.7809387207031 40.35458914848601 39.48988471091403
28
594.1440551757812 40.497918427237785 41.339811777647505
29
348.6343237916825 40.763343092339205 39.199052441489975
30
421.80865182743946 41.10175164491648 41.11278223316427
31
299.68995666503906 39.562612544327365 38.444448798341234
32
402.98866689210575 40.95005762327768 -inf
33
412.03994982551006 40.017892226004854 38.66553113041202
34
306.3025939941406 38.9898

...

751.2457501046034 41.6166641947743 41.363207383581525
25046
451.4961547851562 39.901490376454255 39.343995219496215
25047
417.30569433367106 40.55675832609858 39.81575099074813
25048
245.17151794433593 38.41053139595013 37.7445691896424
25049
681.3130218617475 41.45996567906323 41.744342754005444
25050
399.03252399812504 41.38289938402151 -inf
25051
571.8688397465419 41.50258315157408 41.91724733601218
25052
444.3652465820312 41.12588294193606 41.41008901367049
25053
296.45158081054683 39.899128081545086 38.942224070437355
25054
342.6879608154297 40.5403308091279 39.56815946341745
25055
877.7157470703124 40.921363162233554 39.738470305921375
25056
397.5543701171875 39.67973946318681 39.175201893918654
25057
423.5832275390625 -inf -inf
25058
432.46007266435794 42.22116718908687 -inf
25059
909.4116821289061 -inf -inf
25060
439.9357016549568 41.55232264163486 42.153933128464566
25061
363.3129455566406 40.04238688352385 39.164803277656915
25062
361.25661010742186 40.86677233624883 41.44576

453.1643180804255 40.66686888389867 40.10986932882621
25191
491.0009521484375 40.1443875617694 39.53820108795769
25192
428.81720581054685 40.70088412640188 39.88511856273992
25193
807.5962036132812 41.38453311684324 41.403173684805175
25194
351.62718733148836 41.78046649841668 41.85839438051821
25195
381.2854860180925 41.230222285206146 40.755106527945664
25196
367.5752159092892 41.40740731289311 40.760060558199804
25197
527.8931823730469 41.050484179843295 23.157200875519123
25198
479.00826980068274 40.63814731845213 39.01224160594635
25199
622.5075134277344 40.262597838053814 39.34461151713642
25200
621.8220909149305 41.85550168393863 41.72697784759802
25201
441.2368225097656 41.03883867354238 41.149423883229915
25202
521.4521382756168 40.8658324201228 -inf
25203
571.1049643971174 40.31788841612126 -inf
25204
334.73509521484374 40.02032875518311 39.244193499734465
25205
492.53862792529543 42.112342131251104 42.06435211843047
25206
405.1411554012962 41.54320010333869 41.7886330380467


370.3765625 40.839012455599054 -inf
25333
457.8351623535156 40.19399017021522 39.576582568537155
25334
402.9446329577906 41.04243469033994 -inf
25335
457.64165542414355 41.11993966009238 41.89590825184883
25336
616.0013361023241 41.791174951309856 -inf
25337
715.9688781738281 40.69391605009282 39.75163802243648
25338
902.8514605224141 41.36280747514371 41.611258912415
25339
881.6966095369809 41.48107968096097 41.81752065242264
25340
553.8768798828124 40.395917018174494 38.93818605185927
25341
542.9087807767919 40.93564611368745 -inf
25342
385.2049926757812 40.68576828774219 41.21166453087449
25343
648.7874206542969 40.537472330689845 40.68607894484534
25344
699.802911376953 41.65647589845814 41.58675363467557
25345
432.6475952148437 40.68578508568134 39.59994794077197
25346
866.5400177971204 41.63290159222128 41.108401411795434
25347
425.71024013430156 41.664423477076554 41.63085434821746
25348
385.37482345581884 41.07096534240183 41.28819886734901
25349
586.4084070680316 41.3977588523

254.55001831054685 39.66367742935741 -inf
25477
298.0630889892578 40.64558665647414 40.01824476672977
25478
609.9483581542968 41.46298460451696 41.886067168139476
25479
687.5281830595561 41.11298432259667 41.15582427962954
25480
511.7777160644531 40.4803797288321 41.226987360120596
25481
239.12162780761716 39.768999728466184 -inf
25482
372.8140163989735 41.317206683733204 41.67916450270276
25483
453.69420776367184 41.06472093848865 41.24863762506242
25484
422.7715881347656 40.494893230199644 41.18040160874817
25485
304.8015930175781 38.95972138004382 38.33102845120202
25486
702.7327514648437 41.52811441347362 42.312953895766626
25487
648.611233151928 41.385339702711676 42.09253957152588
25488
298.4926319190692 41.23870158220315 -inf
25489
540.1555060461369 41.07386101335832 40.87425932517046
25490
597.743995281184 41.74821931219009 42.184379660941985
25491
325.69847717285154 39.68488050910637 -inf
25492
782.8987304687499 40.43897957384617 39.992926976024314
25493
410.31887562130225 41.

567.1195434570312 39.86979393801895 39.26980596117332
25622
530.3183166503906 39.657062334131645 39.54770372103901
25623
468.3817877131533 41.69486012131375 24.565437553452465
25624
254.84461186462244 40.459535651140776 -inf
25625
569.0411559507099 41.297598211717386 41.55864948243263
25626
460.38338678341375 41.60887454029826 42.07402772491636
25627
387.01143181956724 40.8293363373788 39.668160007205344
25628
576.6343914253741 41.379501986791134 -inf
25629
746.6305065148614 41.66105556443651 41.525944997540186
25630
291.41644929431556 41.448613334341694 41.81628596050947
25631
578.8356567382812 40.39565259338767 40.19180428981975
25632
390.7628615162665 40.41856203537456 38.87488764940925
25633
285.501626586914 40.38980599877274 40.89636268068917
25634
432.8248596191406 39.371697679397144 38.96337357623558
25635
262.60228028150215 41.15332780859984 40.35083717003097
25636
304.4532434451695 42.36262218872873 42.50144951701047
25637
816.1165846698809 41.95555850369551 41.777245243090334

In [5]:
def script4(input_filename,output_filename,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/Mullaney Spectral Parameters"):
    dir_name = '%s'%(destination_path_cube)   
    os.chdir(dir_name)
    infiles = sorted(glob.glob(input_filename)) # './*source.fits'
    tabledict = {}
  
    for i, file in enumerate(infiles):
        hdulist = fits.open(file)
        bintab = hdulist[1].data
        table = Table(bintab)
        name = table['source no'][0]  
        tabledict[name] = table

    tab = vstack(list(tabledict.values()))
    df = tab.to_pandas()
    df = df.sort_values(by='source no', ascending=True)
    dtab = Table.from_pandas(df)
    dtab.write(output_filename, format='fits', overwrite=True)
    
script4(input_filename='./*source.fits',output_filename='Mullaney.fits')